In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import iqplot

import bokeh.io
import bokeh.models
import bokeh.plotting
import ast
bokeh.io.output_notebook()

In [ ]:
Data = pd.read_csv('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas_Agar/Results/DataSetNew.csv')
Data.head()

In [ ]:
#Data['Face Right']

Data_nostarve = Data[((Data['Training'] == 'Trained') & (Data['Starvation'] == 'Not starved')) | (Data['Training'] == 'Ctrl') ]
Data_nostarve.head()

In [ ]:
#AutoData.columns=AutoData.columns.droplevel(0)
Melted = pd.melt(Data_nostarve,
                 id_vars=['Training', 'Starvation','Reinforced_side'],
                 value_name = 'Count',
                 value_vars=['Peeks Left', 'Peeks Right', 'Peeks Top', 'Long Peeks Left', 'Long Peeks Right', 'Long Peeks Top','Face Left', 'Face Right', 'Face Top'],
                 var_name='Type')

In [ ]:
Melted

In [ ]:
conditions = [
    (Melted['Type'].str.contains('Long')),
    (Melted['Type'].str.contains('Face')),
    (('Long' not in Melted['Type']) & ('Face' not in Melted['Type'])) # This is not really working but still gives expected result
]

values = ['Squeeze', 'Fronts', 'Peeks']
Melted['Beh'] = np.select(conditions,values)
Melted.head()

In [ ]:
conditions = [
    (Melted['Reinforced_side'].str.contains('Right') & (Melted['Type'].str.contains('Right'))),
    (Melted['Reinforced_side'].str.contains('Left') & (Melted['Type'].str.contains('Left'))),
    (Melted['Reinforced_side'].str.contains('Right') & (Melted['Type'].str.contains('Left'))),
    (Melted['Reinforced_side'].str.contains('Left') & (Melted['Type'].str.contains('Right'))),
    (Melted['Type'].str.contains('Top')),
]

values = ['Rewarded Side', 'Rewarded Side', 'Punished Side', 'Punished Side', 'Empty Side']
Melted['Condition'] = np.select(conditions,values)
Melted.head()

In [ ]:
conditions = [
    (Melted['Training'].str.contains('Trained') & (Melted['Starvation'].str.contains('Overnight'))),
    (Melted['Training'].str.contains('Trained') & (Melted['Starvation'].str.contains('Not'))),
    (Melted['Training'].str.contains('Ctrl') & (Melted['Starvation'].str.contains('Overnight'))),
    (Melted['Training'].str.contains('Ctrl') & (Melted['Starvation'].str.contains('Not'))),

]

values = ['Trained Starved', 'Trained', 'Control Starved', 'Control']
Melted['Training_State'] = np.select(conditions,values)
Melted.head()

In [ ]:
g=sns.catplot(x='Condition', y = 'Count',
            hue='Training_State', showfliers = False,
            col='Beh',
            kind="box",
            #row='Starvation',
            data= Melted)
g.map_dataframe(sns.stripplot,x='Condition', y = 'Count',
            hue='Training_State', dodge=True,
            palette=["#404040"])
plt.show

In [ ]:
Melted.Training_State = pd.Categorical(Melted.Training_State,
                                        categories=['Control Starved', 'Trained', ],
                                        ordered=True)

In [ ]:
sns.set(font_scale=1)
g=sns.catplot(x='Condition', y = 'Count',
            hue='Training_State', showfliers = False,
            col='Beh',
            kind="box",
            #row='Starvation',
            data= Melted)
g.map_dataframe(sns.stripplot,x='Condition', y = 'Count',
            hue='Training_State', dodge=True,
            palette=["#404040"])
plt.show

plt.savefig('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas_Agar/Results/Plots/220722_BehaviorPlot.png')

In [ ]:
x = 'Condition'
y = 'Count'
p = bokeh.plotting.figure(
    frame_width=400,
    frame_height=300,
    x_axis_label=x,
    y_axis_label=y,
)

p.box

In [ ]:


p = iqplot.box(
    data=Data,
    q="Count",
    cats="Condition",
)

bokeh.io.show(p)


In [ ]:
for behaviours in np.unique(Melted.Beh):

    print (behaviours)

    p = iqplot.ecdf(
        data=Melted[Melted.Beh == behaviours],
        q='Count',
        style='staircase',
        cats='Training_State',
        #show_legend=False,
        conf_int= True,
    )

    bokeh.io.show(p)

In [ ]:
Grouped = Melted.groupby(Melted.Beh)

In [ ]:
for behaviours in np.unique(Melted.Beh):

    print (behaviours)
    p = iqplot.stripbox(
        data=Melted[Melted.Beh == behaviours],
        q='Count',
        #style='staircase',
        cats='Training_State',
        #show_legend=False,
        box_kwargs=dict(line_width=2),
        whisker_kwargs=dict(line_width=2),
        jitter= True,

    )

    bokeh.io.show(p)

In [ ]:
np.unique(Melted.Beh)

In [ ]:
Data['Peekings_Left'] = sum(1 for i in Data['Durations_Corner_Left'] if i > param)

In [ ]:
Peekparam = 80
SqueezeParam = 160

for index, row in Data.iterrows():
    #print (row['Durations Left Corner'])

    #print (1 for i in row['Durations Left Corner'])
    Data.loc[index,"Peeks Left"] = sum(1 for i in ast.literal_eval(row['Durations Left Corner']) if i > Peekparam)
    Data.loc[index,"Squeeze Left"] = sum(1 for i in ast.literal_eval(row['Durations Left Corner']) if i > SqueezeParam)
    #Data_noWater_Simple['Peeks Left'][rows]= sum(1 for i in Data['Durations Left Corner'][rows] if i > param)
    Data.loc[index,"Peeks Right"] = sum(1 for i in ast.literal_eval(row['Durations Right Corner']) if i > Peekparam)
    Data.loc[index,"Squeeze Right"] = sum(1 for i in ast.literal_eval(row['Durations Right Corner']) if i > SqueezeParam)
    Data.loc[index,"Peeks Top"] = sum(1 for i in ast.literal_eval(row['Durations Top Corner']) if i > Peekparam)
    Data.loc[index,"Squeeze Top"] = sum(1 for i in ast.literal_eval(row['Durations Top Corner']) if i > SqueezeParam)

       # Data_noWater_Simple.apply(sum(1 for i in Data['Durations Left Corner'] if i > param) for elements in Data['Durations Left Corner'])
#Data_noWater_Simple['Peeks Left']

In [ ]:
Melted = pd.melt(Data,
                 id_vars=['Training', 'ObjectsReinforced','Reinforced_side',
                          'Fly', 'Date'],
                 value_name = 'Count',
                 value_vars=['Peeks Left', 'Peeks Right', 'Peeks Top', 'Squeeze Left', 'Squeeze Right', 'Squeeze Top'],
                 var_name='Type')

In [ ]:
conditions = [
    (Melted['Reinforced_side'].str.contains('Right') & (Melted['Type'].str.contains('Right'))),
    (Melted['Reinforced_side'].str.contains('Left') & (Melted['Type'].str.contains('Left'))),
    (Melted['Reinforced_side'].str.contains('Right') & (Melted['Type'].str.contains('Left'))),
    (Melted['Reinforced_side'].str.contains('Left') & (Melted['Type'].str.contains('Right'))),
    (Melted['Type'].str.contains('Top')),
]

values = ['Rewarded', 'Rewarded', 'Punished', 'Punished', 'Empty']
Melted['Condition'] = np.select(conditions,values)
#Melted.head()

conditions = [
    (Melted['Type'].str.contains('Peeks')),
    (Melted['Type'].str.contains('Squeeze')),
]

values = ['Peeking', 'Squeezing',]
Melted['Beh'] = np.select(conditions,values)
#Melted.head()

In [ ]:
sns.set(font_scale=1.5)

g=sns.catplot(x='Condition', y = 'Count',
            hue='Training', showfliers = False,
            col='Beh',
            kind="box",
            #row='Starvation',
            data= Melted)
g.map_dataframe(sns.stripplot,x='Condition', y = 'Count',
            hue='Training', dodge=True,
            palette=["#404040"])
plt.show

plt.savefig('/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/MultiSensory_Project/GatedArenas_Agar/Results/AutoPlot.png')

In [ ]:
sns.set(font_scale=1.5)

g=sns.catplot(x='Condition', y = 'Count',
            hue='Training', showfliers = False,
            col='Beh', row='ObjectsReinforced',
            kind="box",
            #row='Starvation',
            data= Melted)
g.map_dataframe(sns.stripplot,x='Condition', y = 'Count',
            hue='Training', dodge=True,
            palette=["#404040"])
plt.show